# <center><a href="https://t.me/skghd1"><img src='https://upload.wikimedia.org/wikipedia/commons/8/82/Telegram_logo.svg' height="70" alt="Contact using Telegram"/></a><a href="https://github.com/SKGHD/Handy/"><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Font_Awesome_5_brands_github.svg/800px-Font_Awesome_5_brands_github.svg.png' height="70" alt="Github"/></a></center>
<center><img src="https://hitcounter.pythonanywhere.com/count/tag.svg?url=https%3A%2F%2Fgithub.com%2FSKGHD%2FHandy%2F" alt="Hits"></center>
<center><h1><b>HandBrakeCLI-ColabNotebook v1.2</b></h1><center>

Convert/Compress videos using HandBrake in colab.

# GOOGLE DRIVE

In [ ]:
#@title <Center>Mount Google Drive </Center>
#@markdown <Center> <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" alt="Icon">
#@markdown <h6>Use this if and only if you don't use rclone yet.</h6>
#@markdown This may lead to corrupt output file sometimes.
from google.colab import drive
drive.mount('/content/drive')

#Run These first

In [ ]:
#@title <<----<strong>Install HandBrake and rClone</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
HANDBRAKE = True #@param {type:"boolean"}
RCLONE = True #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/content/temp/HandbrakeTemp", exist_ok = True)
makedirs("/root/.config/rclone", exist_ok = True) 
if HANDBRAKE==True:
  !wget -qq https://github.com/vot/ffbinaries-prebuilt/releases/download/v4.2.1/ffmpeg-4.2.1-linux-64.zip 
  !rm -f ffmpeg-4.2.1-linux-64.zip
  !add-apt-repository ppa:stebbins/handbrake-releases -y 
  !apt-get install -y handbrake-cli 
  
if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

In [ ]:
#@title <-----Upload your  Rclone.config file
def moveConfig():
  !mv rclone.conf /root/.config/rclone/rclone.conf

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")

# <center>Rclone Mount/Unmount/Copy </center>
<center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
 
#@markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
 
#@markdown ---
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
mode = "copy" #@param ["copy", "check"]
dry_run = False #@param {type:"boolean"}
Extra_args = "" #@param {type:"string"}
#@markdown ---
Email_notification = False #@param {type:"boolean"}
logs = False #@param {type:"boolean"}
emailID = "" #@param {type:"string"}
password = "" #@param {type:"string"}
Receiver_ID = "" #@param {type:"string"}
#@markdown ---
import smtplib 
########################################
args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
if mode == "check":
  args += "--one-way "
if dry_run == True:
    args +=" --dry-run "
if logs == True:
  args += " --log-file rClone_log.txt "
if Extra_args != "":
  args +=Extra_args
######################################
def runrClone():
  !rclone --user-agent "Mozilla" "$mode" "$source" "$destination" $args
  
def checkEmail():
  if (Email_notification == True and logs == False):
    print("You will receive only a notification after task has finished.")
    runrClone()
    #Send only notification
     
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(emailID , password)
    message = "Your rClone task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)
    print("Email Alert Sent!")  
    s.quit()
  elif (Email_notification == True and logs == True):
    print("You will receive a notification with log attached after task has finished.")
    runrClone()
    #Sending email notification with logs
    
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 

    fromaddr = emailID
    toaddr = Receiver_ID
    msg = MIMEMultipart() 
    # storing the senders email address 
    msg['From'] = fromaddr 
    # storing the receivers email address 
    msg['To'] = toaddr 
    # storing the subject 
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail 
    body = "Your rClone Task has Completed! Visit link to view : https://colab.research.google.com/github/SKGHD/Handy/blob/master/HandBrakeCLI_Colab.ipynb"

    msg.attach(MIMEText(body, 'plain'))
    filename = "rClone_log.txt"
    attachment = open("/content/rClone_log.txt", "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    # encode into base64 
    encoders.encode_base64(p) 
 
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p)  
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, password) 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent with log!")  
    s.quit() 
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    runrClone()
checkEmail()

# <center>***HandBrake*** </center> 
# <Center> <img src="https://handbrake.fr/img/logo.png" width="40"></center>

In [ ]:
#@title <center> HandBrake Configuration </center>
#@markdown > Select Mode ( Batch conversion/ Single File)
MODE = "BATCH" #@param ["SINGLE", "BATCH"]


#@markdown ---
SOURCE = "/content/drives/SKGHD/Anime Downloads/test" #@param {type:"string"}
DESTINATION = "/content/drives/SKGHD/HandBrake" #@param {type:"string"}
FORMAT = "mp4" #@param ["mp4", "mkv"]
RESOLUTION = "480p" #@param ["480p", "720p", "1080p"]
Encoder = "x264" #@param ["x264", "x265"]
Encoder_Preset = "ultrafast" #@param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
#@markdown Choose Constant Quality Rate [Lower = Higher Quality/
#@markdown Larger File Size]
CQ = 30 #@param {type:"slider", min:10, max:30, step:1}
Additional_Flags = "" #@param {type:"string"}
#@markdown ---
Email_Alert = False #@param {type:"boolean"}
MESSAGE = "HandBrakeCLI_Colab has finished working" #@param ["HandBrake has finished working"] {allow-input: true}
SENDER_ID ="" #@param {type:"string"}
SENDER_PASS="" #@param {type:"string"}
RECEIVER_ID="" #@param {type:"string"}
#@markdown >#####  Using an app specific password is recommended: https://myaccount.google.com/apppasswords

########################################################
import smtplib
import os


formats = ('.mkv','.mp4','.ts','.avi','.mov','.wmv')

######## Renames the file ########
def fileName(fPath):
        tName = fPath.split('/')[-1]        
        if tName.endswith('ts'):
          tName = '[HANDY] ' + tName[:-3] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        else:
          tName = '[HANDY] ' + tName[:-4] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        return tName

def set_resolution():
  global w,h,flags
  if RESOLUTION == "480p":
    w, h = "854" , "480"
  elif RESOLUTION == "720p":
    w, h = "1280" , "720"
  elif RESOLUTION=="1080p":
    w, h = "1920" , "1080"

def addFlags():
  global flags
  flags = f" --encoder {Encoder}  --all-audio -s '0,1,2,3' --cfr --optimize --quality={CQ} --width={w} --height={h} --format={FORMAT} --encoder-preset={Encoder_Preset} "
  if Additional_Flags != "":
    flags += str(Additional_Flags)

set_resolution()
addFlags()

##### HandBrake and Rclone #####
def runner(path):
  f_name = fileName(path)
  hTemp=f"/content/temp/HandbrakeTemp/{f_name}"
  !HandBrakeCLI -i "$path" -o "$hTemp" $flags
  
      
  if os.path.isfile(hTemp):
    print(f"\n\n********** Successfully converted {f_name}\n Now saving to Destination.....")
    if os.path.exists('/usr/bin/rclone'):
      !rclone move "$hTemp" --user-agent "Mozilla" "$DESTINATION" --transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40
    else:
      dest = DESTINATION+'/'+f_name
      !mv "$hTemp" "$dest"
  if os.path.isfile(DESTINATION+ '/' +f_name):    
      print(f"\n\n********** Successfully saved {f_name} to Destination")

########## Check Mode ########
if MODE=="BATCH":
  os.makedirs(DESTINATION, exist_ok=True)
  if SOURCE.endswith('/'):
      pass
  else: SOURCE +='/'
  filesList = os.listdir(SOURCE+'.')
  if os.path.isfile(SOURCE+'processed_db.txt'):
    pass
  else:
    with open((SOURCE+'processed_db.txt'), 'w') as fb:
      fb.write("Do not delete this file until all files have been processed!\n")
      fb.close()
  with open((SOURCE+'processed_db.txt'), "r+") as filehandle:
    processedList = [x.rstrip() for x in filehandle.readlines()]

    print('<<<<<<<<<<<<<<<<<< Starting Conversion in Batch mode. >>>>>>>>>>>>>>>>>>')

    for currentFile in filesList:
      if currentFile.endswith(formats):
        if currentFile not in processedList:
          currentPath = SOURCE
          currentPath += currentFile      
          print(f'\n\n**************** Current File to process: {currentFile}')
          #runner(currentPath)
          filehandle.write(currentFile+'\n')
    filehandle.close()
        

else:
    if SOURCE.endswith(formats):    
        runner(SOURCE)
    else: print("Are you sure you have selected the correct file??")

########### Sending Notification ###############
if Email_Alert == True: 
  from email.mime.multipart import MIMEMultipart 
  from email.mime.text import MIMEText 
  msg = MIMEMultipart()
  msg['From'] = SENDER_ID
  msg['To'] = RECEIVER_ID
  msg['Subject'] = "HandBrakeCLI_Colab has finished executing!" 
  
  msg.attach(MIMEText(MESSAGE))
  s = smtplib.SMTP('smtp.gmail.com', 587) 
  s.starttls() 
  s.login(SENDER_ID, SENDER_PASS)
  s.sendmail(SENDER_ID, RECEIVER_ID, msg.as_string())
  print("Email Alert Sent!")  
  s.quit()


# Testing Area(Unfinished)

In [ ]:
 
#@markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
#@markdown <center><b>Unlimited Transfer </b></center> 
#@markdown >##### You'll need to manually enter remote names but this doesn't rely on colab's storage so you can transfer as much as you need without limits.
#@markdown ---
source = "" #@param {type:"string"}
source_remote = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
destination_remote = "" #@param {type:"string"}
mode = "copy" #@param ["copy", "check", "sync"]
dry_run = False #@param {type:"boolean"}
Extra_args = "" #@param {type:"string"}
#@markdown ---
Email_notification = True #@param {type:"boolean"}
logs = False #@param {type:"boolean"}
emailID = "" #@param {type:"string"}
password = "" #@param {type:"string"}
Receiver_ID = "" #@param {type:"string"}
#@markdown ---
import smtplib 
########################################
#args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
args = " --stats-one-line --stats=5s -v "
if mode == "check":
  args += "--one-way "
if dry_run == True:
    args +=" --dry-run "
if logs == True:
  args += " --log-file rClone_log.txt "
if Extra_args != "":
  args +=Extra_args
######################################
def runrClone():
  !rclone --user-agent "Mozilla" "$mode" $source_remote:"$source" $destination_remote:"$destination" $args
  
def checkEmail():
  if (Email_notification == True and logs == False):
    print("You will receive only a notification after task has finished.")
    runrClone()
    #Send only notification
     
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(emailID , password)
    message = "Your rClone task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)
    print("Email Alert Sent!")  
    s.quit()
  elif (Email_notification == True and logs == True):
    print("You will receive a notification with log attached after task has finished.")
    runrClone()
    #Sending email notification with logs
    
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 

    fromaddr = emailID
    toaddr = Receiver_ID
    msg = MIMEMultipart() 
    # storing the senders email address 
    msg['From'] = fromaddr 
    # storing the receivers email address 
    msg['To'] = toaddr 
    # storing the subject 
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail 
    body = "Your rClone Task has Completed! Visit link to view : https://colab.research.google.com/github/SKGHD/Handy/blob/master/HandyCLI.ipynb"

    msg.attach(MIMEText(body, 'plain'))
    filename = "rClone_log.txt"
    attachment = open("/content/rClone_log.txt", "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    # encode into base64 
    encoders.encode_base64(p) 
 
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p)  
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, password) 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent with log!")  
    s.quit() 
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    runrClone()
checkEmail()